# Using VO Tables from HEASARC 

In this notebook, we show how to list, fetch and look at VO tables from the HEASARC archive and also how to create your own VO Table, which can be used to cross-correlate with other VO Tables.  

*  [1. Retrieving](#retrieve) a catalog as a VO Table
*  [2. Creating](#create) a VOTable from scratch
*  [3. Uploading](#upload) our VOTable file for cross-correlation

In [1]:
## The main HTTP request tool we will use:
import requests, io, numpy
## 
from astropy import table as aptable

<a id="retrieve"></a>

# 1. Retrieving a catalog as a VO Table

We can use __[Browse](https://heasarc.gsfc.nasa.gov/cgi-bin/W3Browse/w3browse.pl)__ to get any HEASARC catalog.  Browse can be used to retrieve tables in a number of formats, one of which is a VOTable in XML as defined by the __[IVOA reference](http://www.ivoa.net/documents/VOTable/)__.  

The requests package of python is a generic interface to web services.  The requests.get() method formats the given parameters for you and returns the results of the HTTP request in a useful object.  In this case, the result is a string containing the VOTable XML.  The standard astropy.table package can read these into the standard Table objec.

This fetches the entire rosmaster catalog, so give it a few seconds:

In [2]:
params = {'name': 'rosmaster'}
r = requests.get('http://heasarc.gsfc.nasa.gov/cgi-bin/W3Browse/getvotable.pl', params=params)


The astropy.table package can read the VOTable's XML and create a usual Table object (with some warnings that we can ignore):

In [4]:
table=aptable.Table.read(io.BytesIO(r.content))

In [6]:
table[0:20]

col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,col16,col17,col18,col19,col20,col21,col22,col23,col24,col25,col26,col27,col28
,deg,deg,deg,deg,,,,s,s,,MJD,MJD,,,,,,,,,,,,deg,d,d,
bytes16,float64,float64,float64,float64,bytes6,bytes2,bytes4,int32,int32,bytes10,float64,float64,bytes20,bytes16,bytes14,int32,bytes4,int16,int16,bytes120,int32,int16,int32,int16,int32,int32,int16
RH202299N00,49.3200,-85.5400,299.8517,-30.6815,HRI,N,MPE,36146,70000,RDF 4_2,50174.496192129598,50187.986747685201,RE J0317-853,BURLEIGH,MATTHEW,202299,n00,2,2,RE J0317-853,24302,6,7128,307,2163,2177,2900
RH202299A01,49.3200,-85.5400,299.8517,-30.6815,HRI,N,MPE,43683,70000,RDF 3_6,50324.742534722202,50377.477592592601,RE J0317-853,BURLEIGH,MATTHEW,202299,a01,2,2,RE J0317-853,24301,6,7128,162,2316,2369,2900
RP200594N00,151.8800,-85.0800,299.4192,-23.3707,PSPCB,N,MPE,4231,5000,RDF 3_4,48682.1389583333,48690.983333333301,HD 89499,FLEMING,THOMAS,200594,n00,2,2,HD 89499,17171,2,3532,66,648,657,2900
RH201328N00,107.9100,-84.4700,296.7740,-26.5072,HRI,N,MPE,2332,2000,RDF 3_4,48880.838020833296,48881.780729166698,HD 60102 AB,ZINNECKER,HANS,201328,n00,2,2,HD 60102 AB,22560,3,4268,215,850,851,2900
RH400876A01,193.9400,-83.3700,303.0651,-20.4994,HRI,N,GSFC,19075,20000,RDF 4_2,50702.423530092601,50735.172557870399,2EG J1248-8306 POS 3,HALPERN,JULES,400876,a01,4,2,2EG J1248-8306 POS 3,25116,7,2460,299,2699,2733,9999
RH400876N00,193.9400,-83.3700,303.0651,-20.4994,HRI,N,GSFC,1059,20000,RDF 4_0,50548.2425,50548.255474537,2EG J1248-8306 POS 3,HALPERN,JULES,400876,n00,4,2,2EG J1248-8306 POS 3,19808,7,2460,85,2543,2543,9999
RH400877A02,190.1700,-83.3600,302.6001,-20.4952,HRI,N,GSFC,19384,20000,RDF 4_2,50893.248460648101,50895.138969907399,2EG J1248-8306 POS 4,HALPERN,JULES,400877,a02,4,2,2EG J1248-8306 POS 4,24927,7,2460,90,2893,2895,9999
RH400877A01,190.1700,-83.3600,302.6001,-20.4952,HRI,N,GSFC,2363,20000,RDF 4_2,50735.5647916667,50735.574120370402,2EG J1248-8306 POS 4,HALPERN,JULES,400877,a01,4,2,2EG J1248-8306 POS 4,25115,7,2460,260,2733,2733,9999


The current astroquery.heasarc package has limited functionality, but it will return astropy Tables from object queries to HEASARC catalogs.  (It does not currently use the VO interface.)  For example:

In [7]:
from astroquery.heasarc import Heasarc
heasarc=Heasarc()
heasarc.query_object('3c273',"rosmaster")

ModuleNotFoundError: No module named 'astroquery.heasarc'

<a id="create"></a>
# 2. Creating a VO Table from scratch

Alternatively, we can create a table from scratch using an array of values named whatever we want.  

There are several ways of doing this, and there are a few object layers here, which can be confusing.  There are standard astropy Table objects, there are votable Table objects, and there are votable VOTableFile objects that might contain multiple votable Tables. Though some things can be done with generic astropy Tables, other VO operations can only be done with VO Tables or VOTableFile objects.  

This is easiest to see with an example.  

In [9]:
##  Create a table with only two columns starting from an astropy Table:  
myaptable=aptable.Table(
    numpy.array([
            [19.0186,       46.7304],
            [20.2887,       40.4703],
            [125.886,       21.3377],
            [136.002,       21.9679],
            [141.057,       40.6372],
            [146.700,       22.0116],
            [148.785,       14.2922],
            [149.751,       17.8168],
            [175.039,       15.3270],
            [191.542,       30.7317],
            [194.913,       28.8959],
            [199.026,       41.5011],
            [206.577,       43.8511],
            [209.963,       38.1821],
            [213.556,       15.6214],
            [219.967,       42.7421],
            [226.693,       12.8502],
            [237.489,       20.8057],
            [241.519,       20.8014],
            [317.088,       18.2002],
            [329.235,       6.64845],
            [333.830,       37.3012] ]), 
    names=["ra","dec"])
print(type(myaptable))
myaptable['ra'].format='.4f'
myaptable['dec'].format='.4f'
print(myaptable)


<class 'astropy.table.table.Table'>
   ra      dec  
-------- -------
 19.0186 46.7304
 20.2887 40.4703
125.8860 21.3377
136.0020 21.9679
141.0570 40.6372
146.7000 22.0116
148.7850 14.2922
149.7510 17.8168
175.0390 15.3270
191.5420 30.7317
194.9130 28.8959
199.0260 41.5011
206.5770 43.8511
209.9630 38.1821
213.5560 15.6214
219.9670 42.7421
226.6930 12.8502
237.4890 20.8057
241.5190 20.8014
317.0880 18.2002
329.2350  6.6485
333.8300 37.3012


In [10]:
from astropy.io import votable as apvot

## Then convert this to a VOTableFile object.
myvotablefile = apvot.from_table(myaptable)
print(type(myvotablefile))
## Which contains a nested set of RESOURCES
for r in myvotablefile.resources:
    ## And tables (in this case, only one of each)
    mytable=r
    for t in r.tables:
        print(t)


<class 'astropy.io.votable.tree.VOTableFile'>
        ra                dec        
------------------ ------------------
19.018599999999999 46.730400000000003
20.288699999999999 40.470300000000002
           125.886 21.337700000000002
136.00200000000001            21.9679
141.05699999999999            40.6372
146.69999999999999 22.011600000000001
           148.785 14.292199999999999
           149.751 17.816800000000001
175.03899999999999             15.327
           191.542            30.7317
194.91300000000001 28.895900000000001
199.02600000000001 41.501100000000001
           206.577 43.851100000000002
209.96299999999999 38.182099999999998
213.55600000000001            15.6214
219.96700000000001 42.742100000000001
226.69300000000001 12.850199999999999
           237.489 20.805700000000002
241.51900000000001 20.801400000000001
317.08800000000002 18.200199999999999
329.23500000000001 6.6484500000000004
333.82999999999998 37.301200000000001


<a id="upload"></a>

# 3. Uploading our VOTable file to cross-correlate

The reason we might want not just an astropy Table object but a votable Table object is that we might want to cross-correlate our list of objects with another catalog.  In this case, we would use the __[Table Access Protocol](http://www.ivoa.net/documents/TAP/)__ discussed more extensively in a different notebook.  

Here, we simply show an example of how to hand our VOTable object to the HEASARC TAP service.

We start by writing the votablefile object to a "file-like" object (using BytesIO) that requests will be able to upload (alternatively we could write it to disk, but why?):

In [14]:
## In memory only, use an IO stream. 
vot_obj=io.BytesIO()
apvot.writeto(myvotablefile,vot_obj)
vot_obj.seek(0) # Reset seek to beginning of buffer
## 'uplt' is what we'll call it (for 'upload table') 
##   in the requests parameters below, or what you will:
files={'uplt':vot_obj}

In [12]:
## Alternatively, write it to a file:
#apvot.writeto(votablefile, "new_votable.xml")
#files={'uplt':open('new_votable.xml', 'rb')}

For more information on how to construct the TAP query, see its dedicated notebook.  For our purposes here, this is simply to demonstrate that we can use our VOTable to cross-correlate with, e.g., the CfA Redshift Catalog (zcat).  In other words, the follow retrieves every entry in the zcat that is within a radius of 0.1deg of each of our 30 sources, returning the zcat entries for RA, DEC, and Radial_Velocity.  

(This takes a good 20 seconds. If you need to rerun this cell, rerun the one above that defines the upload.  The IO object needs to be reset.) 

In [15]:
## These parameters are defined in the TAP standard.  
##  The string 'uplt' is what we set above as the name
##  of the parameter containing this data, and
##  the string 'mysources' is how we refer to it in 
##  the ADQL query string:
cc_params={
    'lang': 'ADQL', 
    'request': 'doQuery',
    'upload':'mysources,param:uplt'
    }
cc_params["query"]="""
    SELECT cat.ra, cat.dec, Radial_Velocity 
    FROM zcat cat, tap_upload.mysources mt 
    WHERE
    contains(point('ICRS',cat.ra,cat.dec),circle('ICRS',mt.ra,mt.dec,0.1))=1
    and Radial_Velocity > 0
    ORDER by cat.ra"""
r = requests.post('https://heasarc.gsfc.nasa.gov/xamin/vo/tap/sync',data=cc_params,stream=True,files=files)
mytable=aptable.Table.read(io.BytesIO(r.content))
mytable['ra'].format='.4f'
mytable['dec'].format='.4f'
mytable

ra,dec,radial_velocity
float64,float64,int32
19.0684,46.7400,5081
19.0864,46.7472,5188
20.3348,40.4877,5859
125.9044,21.3382,5219
125.9044,21.3382,5364
135.9942,21.9010,3157
136.0007,21.9679,3093
141.0915,40.6838,8267
146.7033,22.0183,7446
